In [2]:
import torch
import torchaudio
import polars as pl
import matplotlib.pyplot as plt
import os
import numpy as np
import tqdm

import helpers.input_processor as ip

In [3]:
#load data from dataframe
data_dir = "data/raw_training/training_data/"
target_label = 'murmur_in_recording'
df = (
    ip.loadTrainingData(data_dir)
    .filter(pl.col(target_label) != 'Unknown')
    .pipe(ip.encodeData)
    .select([
        pl.col('audio_file').apply(lambda x: os.path.join(data_dir, x)),
        pl.col(target_label)
    ])
)

loading data from save file:  cache/ingested_data.json


In [4]:
#balance the data so that there is an equal number of murmur positive and murmur negative samples
#do this by duplicating random rows of whichever group (pos or neg) is smaller
neg_df = df.filter(pl.col(target_label)==0.0)
pos_df = df.filter(pl.col(target_label)==1.0)
numNeg = neg_df.height
numPos = pos_df.height

while numNeg != numPos:
    if numNeg < numPos:
        df.vstack(neg_df.sample(n=min(numPos-numNeg, neg_df.height), shuffle=True), in_place=True)
    else: 
        df.vstack(pos_df.sample(n=min(numNeg-numPos, pos_df.height), shuffle=True), in_place=True)
    numNeg = df.filter(pl.col(target_label)==0.0).height
    numPos = df.filter(pl.col(target_label)==1.0).height

#reshuffle rows
df = df.sample(frac=1.0, shuffle=True)

#check number of positive and negative samples
numNeg = df.filter(pl.col(target_label)==0.0).height
numPos = df.filter(pl.col(target_label)==1.0).height
print('Total Samples:       ', df.height)
print('Positive Samples:    ', numPos)
print('Negative Samples:    ', numNeg)
print('Percent Positive Samples:    ', numPos/(numPos+numNeg))

Total Samples:        5328
Positive Samples:     2664
Negative Samples:     2664
Percent Positive Samples:     0.5


In [5]:
import librosa
import math, random
import numpy as np

# x = path to audio file
# duration = length of time (in seconds) to which the signal is resized
# sr = sample rate of the signal
# **kwargs = keyword args to pass to librosa.feature.melspectrogram()
def preprocessAudio(x, duration, sr=4000, **kwargs):
    #read and load audio file in .wav format
    sig, samp_rate = librosa.load(x, sr=sr) #make sure that the correct sample rate is passed as a parameters. if unspecified, the function chooses some default value
    
    #resize sample, either by padding it with silence or truncating it
    sig = librosa.util.fix_length(sig, size=sr*duration)

    #implement audio augmentation:  ------------
    #time shift signal to the left or right by a random percent of its original length (max 99%)
    sig_len = sig.shape[0]
    max_shift = 0.99
    sig = np.roll(sig, round(random.random() * max_shift * sig_len))

    # #convert to stereo
    # sig = np.tile(sig, (2,1))
    #-------------------------------------------

    #get Mel spectrogram
    melSpec = librosa.feature.melspectrogram(y=sig, sr=samp_rate)
    melSpec = librosa.amplitude_to_db(melSpec)

    #implement image augmentation   ------------
    #-------------------------------------------

    return melSpec

In [6]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class AudioDataset(Dataset):
    def __init__(self, audioPaths, labels):
        self.audioPaths = audioPaths
        self.labels = labels
        self.sample_duration = 25
        self.n_mels = 128

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        spec = torch.from_numpy(preprocessAudio(self.audioPaths[idx], self.sample_duration, n_mels=self.n_mels))
        label = self.labels[idx]
        return spec, label

In [7]:
def splitDataset(ds, split_ratio=0.8):
    total_size = len(ds)
    train_size = round(split_ratio * total_size)
    test_size = total_size - train_size

    trainSet, testSet = torch.utils.data.random_split(ds, [train_size, test_size], generator=torch.Generator().manual_seed(0))
    return trainSet, testSet

In [8]:

audioPaths = df.get_column('audio_file').to_list()
labels = df.get_column(target_label).to_list()
classes = df.get_column(target_label).unique().to_list()

ds = AudioDataset(audioPaths, labels)
train_ds, test_ds = splitDataset(ds)

# train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
# val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

_________________________________________________________________________________________________________
tutorial code
_________________________________________________________________________________________________________

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
num_epochs = 5
batch_size = 32
learning_rate = 0.001

# Dataloaders
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)
        numClasses = 2
        self.conv1 = nn.Conv2d(1, 2*1, 5)   # first param = 1 since iput image has 1 channel
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(2*1, 16, 5)
        self.fc1 = nn.Linear(16 * 29 * 46, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, numClasses)


    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 29 * 46)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x


model = ConvNet().to(device)

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    running_loss = 0.0
    epoch_bar = tqdm.tqdm(train_loader)
    epoch_bar.set_description("Epoch %s" % epoch)
    for i, (images, labels) in enumerate(epoch_bar):
        epoch_bar.set_postfix({"loss": running_loss})

        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.type(torch.LongTensor).to(device)

        # Forward pass
        outputs = model(images.unsqueeze(1))
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss = loss.item()

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch 4: 100%|██████████| 134/134 [01:02<00:00,  2.16it/s, loss=0.626]

Finished Training


In [19]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(2)]
    n_class_samples = [0 for i in range(2)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images.unsqueeze(1))
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = int(labels[i].item())
            pred = int(predicted[i].item())
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

IndexError: index 10 is out of bounds for dimension 0 with size 10